In [1]:
# LIBRAIRES
import numpy as np
import pandas as pd
import random
import pickle

import string

from matplotlib import pyplot as plt 

In [2]:
# METHODS
def swap_vals(dirty_df, n):
    mods = pd.DataFrame()
    for i in range(n):
        col_i = random.randint(0,dirty_df.shape[1]-1)
        col = dirty_df.columns[col_i]
        row_1 = random.randint(0,dirty_df.shape[0]-1)
        row_2 = random.randint(0,dirty_df.shape[0]-1)
        # val = dirty_df.iloc[row][col]
        val_1 = dirty_df.iat[row_1,col_i]
        val_2 = dirty_df.iat[row_2,col_i]
    
        dirty_df.iat[row_1,col_i] = val_2
        dirty_df.iat[row_2,col_i] = val_1
    
        mod = {'col': col, 'row 1': row_1, 'val 1': val_1,'row 2': row_2,'val 2':val_2}
        # mods = mods.append(mod, ignore_index = True)
        mods = pd.concat([mods, pd.DataFrame([mod])], ignore_index=True)
    return(dirty_df,mods)

def rand_mod(dirty_df, n):
    mods = pd.DataFrame()
    for i in range(n):
        col_i = random.randint(0,dirty_df.shape[1]-1)
        col = dirty_df.columns[col_i]
        row_1 = random.randint(0,dirty_df.shape[0]-1)
        row_2 = random.randint(0,dirty_df.shape[0]-1)
        # val = dirty_df.iloc[row][col]
        val_1 = dirty_df.iat[row_1,col_i]
        val_2 = dirty_df.iat[row_2,col_i]
    
        dirty_df.iat[row_1,col_i] = val_2
        dirty_df.iat[row_2,col_i] = val_1
    
        mod = {'col': col, 'row 1': row_1, 'val 1': val_1,'row 2': row_2,'val 2':val_2}
        # mods = mods.append(mod, ignore_index = True)
        mods = pd.concat([mods, pd.DataFrame([mod])], ignore_index=True)

    return(dirty_df,mods)

In [3]:
# GET DATA
df = pd.read_csv('datasets/compas/compas-scores.csv')
df.shape

(11757, 47)

In [4]:
# CLEAN SOME DATATYPES
df["compas_screening_date"] = df["compas_screening_date"].astype("datetime64[ns]")
df["c_jail_in"] = df["c_jail_in"].astype("datetime64[ns]")
df["c_jail_out"] = df["c_jail_out"].astype("datetime64[ns]")
df["c_offense_date"] = df["c_offense_date"].astype("datetime64[ns]")
df["r_offense_date"] = df["r_offense_date"].astype("datetime64[ns]")
df["c_arrest_date"] = df["c_arrest_date"].astype("datetime64[ns]")
df["vr_offense_date"] = df["vr_offense_date"].astype("datetime64[ns]")
df["v_screening_date"] = df["v_screening_date"].astype("datetime64[ns]")

df.dtypes

id                                  int64
name                               object
first                              object
last                               object
compas_screening_date      datetime64[ns]
sex                                object
dob                                object
age                                 int64
age_cat                            object
race                               object
juv_fel_count                       int64
decile_score                        int64
juv_misd_count                      int64
juv_other_count                     int64
priors_count                        int64
days_b_screening_arrest           float64
c_jail_in                  datetime64[ns]
c_jail_out                 datetime64[ns]
c_case_number                      object
c_offense_date             datetime64[ns]
c_arrest_date              datetime64[ns]
c_days_from_compas                float64
c_charge_degree                    object
c_charge_desc                     

In [5]:
print(df.race.unique())
map = {
    'Caucasian': 0,
    'African-American': 1,
    'Hispanic': 1,
    'Asian': 1,
    'Native American': 1,
    'Other': 1
}

Y = df['is_recid'].values
P_val = df['race'].values
P = df['race'].map(map).values

df = df.drop(columns=['is_recid']) # remove label

['Other' 'Caucasian' 'African-American' 'Hispanic' 'Asian'
 'Native American']


In [6]:
df_dirty = df.copy()
df_dirty,swap_mods=swap_vals(df_dirty, 1000)
df_dirty,rand_mods=rand_mod(df_dirty, 5000)
# mods

In [7]:
data_dict = {
    'Y':Y,
    'P_val':P_val,
    'P':P
}
file = open('datasets/compas/data.p', 'wb')
pickle.dump(data_dict,file)
fileObj.close()

df.to_csv("datasets/compas/clean.csv", index=False)
df_dirty.to_csv("datasets/compas/dirty.csv", index=False)

NameError: name 'fileObj' is not defined